# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [4]:
#Passo 1: entrar na Internet
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

navegador = webdriver.Chrome()

#Passo 2: pegar a cotação do dólar
navegador.get('https://www.google.com/')
navegador.find_element_by_xpath(
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('Cotação dólar')

navegador.find_element_by_xpath(
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_dolar = navegador.find_element_by_xpath(
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacao_dolar)
    
#Passo 3: pegar a cotação do euro
navegador.get('https://www.google.com/')
navegador.find_element_by_xpath(
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('Cotação euro')

navegador.find_element_by_xpath(
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element_by_xpath(
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacao_euro)

#Passo 4: pegar a cotação do OURO
navegador.get('https://www.melhorcambio.com/ouro-hoje')
cotacao_ouro = navegador.find_element_by_xpath('//*[@id="comercial"]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace(",",".")
print(cotacao_ouro)

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 92
Current browser version is 96.0.4664.45 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [ ]:
#Passo 5: importar e atualizar a base de dados
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

- Atualizando os preços e o cálculo do Preço Final

In [ ]:
# Atualizar a cotação
tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)

tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)

tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

#Atualizar o preço de compra -> Preço original * Cotação
tabela["Preço Base Reais"] = tabela["Preço Base Original"] * tabela["Cotação"]

#Atualizar o preço de venda -> Preço de compra * Margem
tabela["Preço Final"] = tabela["Preço Base Reais"] * tabela["Margem"]

tabela["Preço Final"] = tabela["Preço Final"].map("{:.2f}".format)

display(tabela)

### Agora vamos exportar a nova base de preços atualizada

In [ ]:
#Passo 6: Exportar a base de dados atualizada
tabela.to_excel("Produtos_Novo.xlsx", index=False)